# Quick Start Guide 🚀

This notebook goes the basic components of the `fastplotlib` API, image, image updates, line plots, and scatter plots.

**NOTE: This quick start guide in the docs is NOT interactive. Download the examples from the repo and try them on your own computer. You can run the desktop examples directly if you have `glfw` installed, or try the notebook demos:** https://github.com/kushalkolar/fastplotlib/tree/master/examples

It will not be possible to have live demos on the docs until someone can figure out how to get [pygfx](https://github.com/pygfx/pygfx) to work with `wgpu` in the browser, perhaps through [pyodide](https://github.com/pyodide/pyodide) or something :D.

In [ ]:
import fastplotlib as fpl
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

In [ ]:
import os

## Images

In [ ]:
# create a `Plot` instance
plot = fpl.Plot()

# make some random 2D image data
data = np.random.rand(512, 512)

# plot the image data
image_graphic = plot.add_image(data=data, name="random-image")

# show the plot
plot.show()

In live notebooks or desktop applications, you can use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!

Changing graphic "features"

In [ ]:
image_graphic.cmap = "viridis"

This is how you can take a snapshot of the canvas. Snapshots are shown throughout this doc page for the purposes of documentation, they are NOT necessary for real interactive usage. Download the notebooks to run live demos.

In [ ]:
plot.canvas.snapshot()

Setting image data

In [ ]:
image_graphic.data = 0

In [ ]:
plot.canvas.snapshot()

Setting image data with slicing

In [ ]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

In [ ]:
plot.canvas.snapshot()

Setting image data back to random

In [ ]:
image_graphic.data = np.random.rand(512, 512)

In [ ]:
plot.canvas.snapshot()

Plots are indexable and give you their graphics by name

In [ ]:
plot.graphics

In [ ]:
plot["random-image"]

The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`.

In [ ]:
image_graphic

In [ ]:
image_graphic == plot["random-image"]

## Image updates

This examples show how you can define animation functions that run on every render cycle.

In [ ]:
# create another `Plot` instance
plot_v = fpl.Plot()

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an arugment
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# show the plot
plot_v.show()

**Share controllers across plots**

This example creates a new plot, but it synchronizes the pan-zoom controller

In [ ]:
plot_sync = fpl.Plot(controller=plot_v.controller)

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show()

Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

You can also use `ipywidgets.VBox` and `HBox` to stack plots. See the `gridplot` notebooks for a proper gridplot interface for more automated subplotting

Not shown in the docs, try the live demo for this feature

In [ ]:
#VBox([plot_v.canvas, plot_sync.show()])

In [ ]:
#HBox([plot_v.show(), plot_sync.show()])

## Line plots

2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how **Graphic Features** can be modified by slicing!

Generate some data.

In [ ]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

Plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [ ]:
# Create a plot instance
plot_l = fpl.Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

plot_l.show()

"stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the **`1:1`** button to toggle this.

In [ ]:
plot_l.camera.maintain_aspect = False

reset the plot area

In [ ]:
plot_l.auto_scale(maintain_aspect=True)

Graphic features support slicing! :D 

In [ ]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"
cosine_graphic.colors[90:] = "red"
cosine_graphic.colors[60] = "w"

# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

Make a snapshot of the canvas after slicing

In [ ]:
plot_l.canvas.snapshot()

**You can capture changes to a graphic feature as events**

In [ ]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [ ]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

In [ ]:
plot_l.canvas.snapshot()

Graphic `data` is also indexable

In [ ]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]
cosine_graphic.data[90:, 1] = 7

In [ ]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

In [ ]:
plot_l.canvas.snapshot()

Toggle the presence of a graphic within the scene

In [ ]:
sinc_graphic.present = False

In [ ]:
plot_l.canvas.snapshot()

In [ ]:
sinc_graphic.present = True

In [ ]:
plot_l.canvas.snapshot()

You can create callbacks to `present` too, for example to re-scale the plot w.r.t. graphics that are present in the scene

In [ ]:
sinc_graphic.present.add_event_handler(plot_l.auto_scale)

In [ ]:
sinc_graphic.present = False

In [ ]:
plot_l.canvas.snapshot()

In [ ]:
sinc_graphic.present = True

In [ ]:
plot_l.canvas.snapshot()

You can set the z-positions of graphics to have them appear under or over other graphics

In [ ]:
img = np.random.rand(20, 100)

plot_l.add_image(img, name="image", cmap="gray")

# z axix position -1 so it is below all the lines
plot_l["image"].position_z = -1
plot_l["image"].position_x = -50

In [ ]:
plot_l.canvas.snapshot()

### 3D line plot

In [ ]:
# just set the camera as "3d", the rest is basically the same :D 
plot_l3d = fpl.Plot(camera='3d')

# create a spiral
phi = np.linspace(0, 30, 200)

xs = phi * np.cos(phi)
ys = phi * np.sin(phi)
zs = phi

# use 3D data
# note: you usually mix 3D and 2D graphics on the same plot
spiral = np.dstack([xs, ys, zs])[0]

plot_l3d.add_line(data=spiral, thickness=2, cmap='winter')

plot_l3d.show()

In [ ]:
plot_l3d.auto_scale(maintain_aspect=True)

## Scatter plots

Plot tens of thousands or millions of points

There might be a small delay for a few seconds before the plot shows, this is due to shaders being compiled and a few other things. The plot should be very fast and responsive once it is displayed and future modifications should also be fast!

In [ ]:
# create a random distribution
# only 1,000 points shown here in the docs, but it can be millions
n_points = 1_000

# if you have a good GPU go for 1.5 million points :D 
# this is multiplied by 3
#n_points = 500_000

# dimensions always have to be [n_points, xyz]
dims = (n_points, 3)

clouds_offset = 15

# create some random clouds
normal = np.random.normal(size=dims, scale=5)
# stack the data into a single array
cloud = np.vstack(
    [
        normal - clouds_offset,
        normal,
        normal + clouds_offset,
    ]
)

# color each of them separately
colors = ["yellow"] * n_points + ["cyan"] * n_points + ["magenta"] * n_points

# create plot
plot_s = fpl.Plot()

# use an alpha value since this will be a lot of points
scatter_graphic = plot_s.add_scatter(data=cloud, sizes=3, colors=colors, alpha=0.7)

plot_s.show()

**Scatter graphic features work similarly to line graphic**

In [ ]:
# half of the first cloud's points to red
scatter_graphic.colors[:n_points:2] = "r"

In [ ]:
plot_s.canvas.snapshot()

In [ ]:
# set the green value directly
scatter_graphic.colors[n_points:n_points * 2, 1] = 0.3

In [ ]:
plot_s.canvas.snapshot()

In [ ]:
# set color values directly using an array
scatter_graphic.colors[n_points * 2:] = np.repeat([[1, 1, 0, 0.5]], n_points, axis=0)

In [ ]:
plot_s.canvas.snapshot()

In [ ]:
# change the data, change y-values
scatter_graphic.data[n_points:n_points * 2, 1] += 15

In [ ]:
plot_s.canvas.snapshot()

In [ ]:
# set x values directly but using an array
scatter_graphic.data[n_points:n_points * 2, 0] = np.linspace(-40, 0, n_points)

In [ ]:
plot_s.canvas.snapshot()

## ipywidget layouts

This just plots everything from these examples in a single output cell

In [ ]:
# row1 = HBox([plot.show(), plot_v.show(), plot_sync.show()])
# row2 = HBox([plot_l.show(), plot_l3d.show(), plot_s.show()])

# VBox([row1, row2])

## Gridplot

Subplots within a `GridPlot` behave the same as simple `Plot` instances! 

💡 `Plot` is actually a subclass of `Subplot`!

In [ ]:
# GridPlot of shape 2 x 3 with all controllers synced
grid_plot = fpl.GridPlot(shape=(2, 3), controllers="sync")

# Make a random image graphic for each subplot
for subplot in grid_plot:
    # create image data
    data = np.random.rand(512, 512)
    # add an image to the subplot
    subplot.add_image(data, name="rand-img")

# Define a function to update the image graphics with new data
# add_animations will pass the gridplot to the animation function
def update_data(gp):
    for sp in gp:
        new_data = np.random.rand(512, 512)
        # index the image graphic by name and set the data
        sp["rand-img"].data = new_data
        
# add the animation function
grid_plot.add_animations(update_data)

# show the gridplot 
grid_plot.show()

### Slicing GridPlot

In [ ]:
# positional indexing
# row 0 and col 0
grid_plot[0, 0]

You can get the graphics within a subplot, just like with simple `Plot`

In [ ]:
grid_plot[0, 1].graphics

and change their properties

In [ ]:
grid_plot[0, 1].graphics[0].vmax = 0.5

more slicing with `GridPlot`

In [ ]:
# you can give subplots human-readable string names
grid_plot[0, 2].name = "top-right-plot"

In [ ]:
grid_plot["top-right-plot"]

In [ ]:
# view its position
grid_plot["top-right-plot"].position

In [ ]:
# these are really the same
grid_plot["top-right-plot"] is grid_plot[0, 2]

Indexing with subplot name and graphic name

In [ ]:
grid_plot["top-right-plot"]["rand-img"].vmin = 0.5

## GridPlot customization

In [ ]:
# grid with 2 rows and 3 columns
grid_shape = (2, 3)

# pan-zoom controllers for each view
# views are synced if they have the 
# same controller ID
controllers = [
    [0, 3, 1],  # id each controller with an integer
    [2, 2, 3]
]


# you can give string names for each subplot within the gridplot
names = [
    ["subplot0", "subplot1", "subplot2"],
    ["subplot3", "subplot4", "subplot5"]
]

# Create the grid plot
grid_plot = fpl.GridPlot(
    shape=grid_shape,
    controllers=controllers,
    names=names,
)


# Make a random image graphic for each subplot
for subplot in grid_plot:
    data = np.random.rand(512, 512)
    # create and add an ImageGraphic
    subplot.add_image(data=data, name="rand-image")
    

# Define a function to update the image graphics 
# with new randomly generated data
def set_random_frame(gp):
    for subplot in gp:
        new_data = np.random.rand(512, 512)
        subplot["rand-image"].data = new_data

# add the animation
grid_plot.add_animations(set_random_frame)
grid_plot.show()

Indexing the gridplot to access subplots

In [ ]:
# can access subplot by name
grid_plot["subplot0"]

In [ ]:
# can access subplot by index
grid_plot[0, 0]

**subplots also support indexing!**

this can be used to get graphics if they are named

In [ ]:
# can access graphic directly via name
grid_plot["subplot0"]["rand-image"]

In [ ]:
grid_plot["subplot0"]["rand-image"].vmin = 0.6
grid_plot["subplot0"]["rand-image"].vmax = 0.8

positional indexing also works event if subplots have string names

In [ ]:
grid_plot[1, 0]["rand-image"].vim = 0.1
grid_plot[1, 0]["rand-image"].vmax = 0.3

In [ ]:
grid_plot[1, 0]["rand-image"].type